In [1]:
import cv2
import numpy as np

def preprocess_image_file(image_path, target=(100, 100)):
    # Load the image
    img = cv2.imread(image_path)
    
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    
    # Get original dimensions
    h, w = img.shape[:2]
    
    # Center crop (crop to the smallest dimension to make it square)
    min_dim = min(h, w)
    start_x = (w - min_dim) // 2
    start_y = (h - min_dim) // 2
    img = img[start_y:start_y + min_dim, start_x:start_x + min_dim]
    
    # Resize to 100x100
    img = cv2.resize(img, target)
    
    # # Convert grayscale to RGB if needed
    # if len(img.shape) == 2:  # Grayscale image
    #     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
    # # Convert RGBA to RGB if needed
    # if img.shape[2] == 4:  # RGBA image
    #     img = img[:, :, :3]
    
    # Normalize pixel values to [0,1]
    img = img / 255.0

    # Flatten to a 1D vector of length 30,000
    img = img.flatten()
    
    return img

In [2]:
import os

def preprocess_image(image_dir, class_label):
    # returns X and Y numpy arrays of images and labels
    # skip = 0
    X = []
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for valid image files
            image_path = os.path.join(image_dir, filename)
            # Resize and center crop the image
            try:
                image = preprocess_image_file(image_path)
                X.append(image)
            except ValueError as e:
                print(e)

    # Create a NumPy array from the list of images
    X = np.array(X)
    Y = np.full((X.shape[0], 1), class_label)  # Create an array of labels
    return X, Y

In [ ]:
train_dir0 = "../data/Q2/train/dew"
train_dir1 = "../data/Q2/train/fogsmog"

x0, y0 = preprocess_image(train_dir0, 0)
x1, y1 = preprocess_image(train_dir1, 1)
X_train = np.concatenate((x1, x0), axis=0)
Y_train = np.concatenate((y1, y0), axis=0).ravel()

test_dir0 = "../data/Q2/test/dew"
test_dir1 = "../data/Q2/test/fogsmog"

x2, y2 = preprocess_image(test_dir0, 0)
x3, y3 = preprocess_image(test_dir1, 1)
X_test = np.concatenate((x3, x2), axis=0)
Y_test = np.concatenate((y3, y2), axis=0).ravel()

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
from svm import SupportVectorMachine
import time
# Initialize the SVM with a linear kernel
svm_linear = SupportVectorMachine()

# Train the SVM with C = 1.0
start = time.time()
svm_linear.fit(X_train, Y_train, kernel='linear', C=1.0)
end = time.time()
train_time_lin = end - start
print(f"Training time: {train_time_lin:.4f} seconds")

# Number of support vectors
num_support_vectors = len(svm_linear.svalphas)
percentage_support_vectors = (num_support_vectors / X_train.shape[0]) * 100

print(f"Number of support vectors: {num_support_vectors}")
print(f"Percentage of training samples that are support vectors: {percentage_support_vectors:.2f}%")

In [ ]:
# Predict on the test set
Y_pred_linear = svm_linear.predict(X_test)

# Calculate test accuracy
test_accuracy_linear = np.mean(Y_pred_linear == Y_test) * 100
print(f"Test set accuracy (Linear Kernel): {test_accuracy_linear:.2f}%")

# Weight vector and intercept term
print(f"Weight vector (w): {svm_linear.w}")
print(f"Intercept term (b): {svm_linear.b}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get indices of top support vectors
sorted_indices = np.argsort(svm_linear.svalphas)[::-1]  # Sort in descending order
all_sv_images = svm_linear.svX[sorted_indices].reshape(-1, 100, 100, 3)

# Filter unique support vectors
unique_sv = []
seen_hashes = set()

for i, sv in enumerate(all_sv_images):
    sv_flattened = tuple(sv.flatten())  # Convert to tuple for hashability
    if sv_flattened not in seen_hashes:
        seen_hashes.add(sv_flattened)
        unique_sv.append((sorted_indices[i], sv))  # Keep index for reference
    if len(unique_sv) == 5:  # Stop when we have 5 unique ones
        break

# Extract indices and images of unique support vectors
top_5_indices, top_5_sv_images = zip(*unique_sv)

# Compute weight vector image
w_image = svm_linear.w.reshape(100, 100, 3)
w_image = (w_image - w_image.min()) / (w_image.max() - w_image.min())  # Normalize

# Plot support vectors and weight vector
fig, axes = plt.subplots(1, 6, figsize=(15, 5))

for i in range(len(top_5_sv_images)):
    axes[i].imshow(top_5_sv_images[i])
    axes[i].set_title(f"Support Vector {i+1}")
    axes[i].axis("off")

# Plot the weight vector w as an image
axes[5].imshow(w_image)
axes[5].set_title("Weight Vector w")
axes[5].axis("off")

In [ ]:
from svm import SupportVectorMachine
# Initialize the SVM with a linear kernel
svm_gaussian = SupportVectorMachine()

# Train the SVM with C = 1.0
start = time.time()
svm_gaussian.fit(X_train, Y_train, kernel='gaussian', C=1.0, gamma=0.001)
end = time.time()
train_time_gaussian = end - start
print(f"Training time: {train_time_gaussian:.4f} seconds")

# Number of support vectors
num_support_vectors = len(svm_gaussian.svalphas)
percentage_support_vectors = (num_support_vectors / X_train.shape[0]) * 100

print(f"Number of support vectors: {num_support_vectors}")
print(f"Percentage of training samples that are support vectors: {percentage_support_vectors:.2f}%")

In [ ]:
# Predict on the test set
Y_pred_gaussian = svm_gaussian.predict(X_test)

# Calculate test accuracy
test_accuracy_gaussian = np.mean(Y_pred_gaussian == Y_test) * 100
print(f"Test set accuracy (Gaussian Kernel): {test_accuracy_gaussian:.2f}%")

# Weight vector and intercept term
print(f"Intercept term (b): {svm_gaussian.b}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

top_5_indices = np.argsort(svm_gaussian.svalphas)[-5:][::-1]
top_5_sv_images = svm_gaussian.svX[top_5_indices].reshape(-1, 100, 100, 3)
duplicates = np.sum(np.abs(top_5_sv_images[0] - top_5_sv_images[1]))

fig, axes = plt.subplots(1, 5, figsize=(15, 5))

for i in range(5):
    axes[i].imshow(top_5_sv_images[i])
    axes[i].set_title(f"Support Vector {i+1}")
    axes[i].axis("off")

plt.show()

In [ ]:
import numpy as np

# Extract support vectors
sv_linear = svm_linear.svX  # Support vectors from linear SVM
sv_gaussian = svm_gaussian.svX  # Support vectors from Gaussian SVM

# Check for common support vectors
common_sv_count = sum(np.any(np.all(sv_linear[:, None] == sv_gaussian, axis=-1), axis=0))

print(f"Number of common support vectors: {common_sv_count}")

In [ ]:
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVM with a linear kernel
svm_sklearn_linear = SVC(kernel='linear', C=1.0)
start = time.time()
# Train the SVM with C = 1.0
svm_sklearn_linear.fit(X_train, Y_train)
end = time.time()

# Predictions
y_pred_lin = svm_sklearn_linear.predict(X_test)
test_accuracy = accuracy_score(Y_test, y_pred_lin)

print("SVMLinear Stats")
print(f"Training time: {end - start:.2f}s")
print(f"Number of Support Vectors: {len(svm_sklearn_linear.support_)}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
w_sklearn = np.dot(svm_sklearn_linear.dual_coef_, svm_sklearn_linear.support_vectors_).flatten()
b_sklearn = svm_sklearn_linear.intercept_[0]
print(f"Sklearn SVM - w : {w_sklearn}")
print(f"Sklearn SVM - w norm: {np.linalg.norm(w_sklearn)}")
print(f"Sklearn SVM - bias (b): {b_sklearn}")
sv_sklearn = svm_sklearn_linear.support_vectors_  # Support vectors from Scikit-learn
matching_sv_count_1 = sum(np.any(np.all(sv_linear[:, None] == sv_sklearn, axis=-1), axis=0))
print(f"Number of common support vectors (with CVXOPT linear): {matching_sv_count_1}")
matching_sv_count_2 = sum(np.any(np.all(sv_gaussian[:, None] == sv_sklearn, axis=-1), axis=0))
print(f"Number of common support vectors (with CVXOPT Gaussian): {matching_sv_count_2}")

In [ ]:
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVM with a Gaussian (RBF) kernel
svm_sklearn_rbf = SVC(kernel='rbf', C=1.0, gamma=0.001)
start = time.time()
# Train the SVM
svm_sklearn_rbf.fit(X_train, Y_train)
end = time.time()

# Predictions
y_pred_rbf = svm_sklearn_rbf.predict(X_test)
test_accuracy_rbf = accuracy_score(Y_test, y_pred_rbf)

print("\nSVMGaussian Stats")
print(f"Training time: {end - start:.2f}s")
print(f"Number of Support Vectors: {len(svm_sklearn_rbf.support_)}")
print(f"Test Accuracy: {test_accuracy_rbf * 100:.2f}%")

# Extract support vectors
sv_sklearn_rbf = svm_sklearn_rbf.support_vectors_

# Compare with CVXOPT support vectors
matching_sv_count_1 = sum(np.any(np.all(sv_linear[:, None] == sv_sklearn_rbf, axis=-1), axis=0))
print(f"Number of common support vectors (with CVXOPT linear): {matching_sv_count_1}")
matching_sv_count_2 = sum(np.any(np.all(sv_gaussian[:, None] == sv_sklearn_rbf, axis=-1), axis=0))
print(f"Number of common support vectors (with CVXOPT Gaussian): {matching_sv_count_2}")

# Bias term
b_sklearn_rbf = svm_sklearn_rbf.intercept_[0]
print(f"Sklearn SVM - bias (b): {b_sklearn_rbf}")

In [ ]:
import numpy as np
import time
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# Initialize the SGDClassifier with a hinge loss (which is equivalent to SVM)
sgd_svm = SGDClassifier(loss='hinge', alpha=1e-4, max_iter=1000, tol=1e-4, random_state=42)

# Train and measure time
start = time.time()
sgd_svm.fit(X_train, Y_train)
end = time.time()

# Predictions and accuracy
y_pred_sgd = sgd_svm.predict(X_test)
test_accuracy_sgd = accuracy_score(Y_test, y_pred_sgd)

print("\nSGD SVM Stats")
print(f"Training time: {end - start:.2f}s")
print(f"Number of Iterations: {sgd_svm.n_iter_}")
print(f"Test Accuracy: {test_accuracy_sgd * 100:.2f}%")

# Extract weight vector and bias
w_sgd = sgd_svm.coef_.flatten()
b_sgd = sgd_svm.intercept_[0]
print(f"SGD SVM - w norm: {np.linalg.norm(w_sgd)}")
print(f"SGD SVM - bias (b): {b_sgd}")

**ONE-VS-ONE MULTICLASS CLASSIFIER**

In [3]:
def load_data(base_path):
    classes = os.listdir(base_path)  # Maintain original order
    class_map = {cls: i for i, cls in enumerate(classes)}  # Map class names to indices
    class_dirs = {i: os.path.join(base_path, cls) for cls, i in class_map.items()}  # Use class name for path

    # Count number of images in each class directory
    # class_counts = {i: len(os.listdir(class_dirs[i])) for i in class_dirs}
    # for i, count in class_counts.items():
    #     print(f"Class {i} ({list(class_map.keys())[i]}): {count} images")
    
    return class_dirs, class_map


In [4]:
train_dirs, train_map = load_data("../data/Q2/train")
test_dirs, test_map = load_data("../data/Q2/test")

In [ ]:
train_data = {}
for i in range(11):
    train_data[f"x{i}_0"], train_data[f"y{i}_0"] = preprocess_image(train_dirs[i], 0)
    train_data[f"x{i}_1"], train_data[f"y{i}_1"] = preprocess_image(train_dirs[i], 1)

# print(train_data.keys())

# Print number of images per class variation
# for i in range(11):
#     print(f"Class {i}: x{i}_0 - {len(train_data[f'x{i}_0'])} images, x{i}_1 - {len(train_data[f'x{i}_1'])} images")

In [ ]:
from svm import SupportVectorMachine
from itertools import combinations
svm_models = {}
for i, j in combinations(range(11), 2):
    X_train = np.concatenate((train_data[f"x{i}_0"], train_data[f"x{j}_1"]), axis=0)
    Y_train = np.concatenate((train_data[f"y{i}_0"], train_data[f"y{j}_1"]), axis=0).ravel()
    
    model_name = f"Bin_SVM_{i}_{j}"
    svm_models[(i,j)] = SupportVectorMachine()
    print(f"Training {model_name}...")
    svm_models[(i,j)].fit(X_train, Y_train, kernel='gaussian', C=1.0, gamma=0.001)
    print(f"Training complete for {model_name}")

In [ ]:
import numpy as np

# Initialize empty arrays
X_test, Y_test = None, None

for i in range(11):  # Assuming 11 classes
    X, Y = preprocess_image(test_dirs[i], i)  # Get processed images and labels

    # Concatenate iteratively
    if X_test is None:
        X_test, Y_test = X, Y  # First batch initializes arrays
    else:
        X_test = np.concatenate((X_test, X), axis=0)
        Y_test = np.concatenate((Y_test, Y), axis=0)

# Flatten labels to 1D array
Y_test = Y_test.ravel()

# print(X_test.shape, Y_test.shape)
    

In [ ]:
import numpy as np

num_classes = 11  # Total number of classes
num_samples = X_test.shape[0]  # Number of test images

# Step 1: Initialize vote count and score sum arrays
vote_counts = np.zeros((num_samples, num_classes), dtype=int)  # Shape: (num_samples, 11)
score_sums = np.zeros((num_samples, num_classes), dtype=float)  # Shape: (num_samples, 11)

# Step 2: Iterate through all trained SVM classifiers
for (i, j), model in svm_models.items():
    predictions, decision_values = model.predict_gaussian(X_test)  # Get decision values for all test samples

    # Determine class assignments
    pred_classes = np.where(predictions == 1, j, i)  # Assign class j if 1, else class i

    # Update vote counts and score sums
    for k in range(num_samples):
        vote_counts[k, pred_classes[k]] += 1
        score_sums[k, pred_classes[k]] += abs(decision_values[k])

# Step 3: Determine the final predicted class for each test sample
max_votes = np.max(vote_counts, axis=1, keepdims=True)  # Shape: (num_samples, 1)
candidates = (vote_counts == max_votes)  # Boolean mask for classes with max votes

# Resolve ties using score sums
final_predictions = np.argmax(candidates * score_sums, axis=1)

print(f"Final predicted classes: {final_predictions}")
# Calculate test accuracy
test_accuracy_multi = np.mean(final_predictions == Y_test) * 100
print(f"Test set accuracy: {test_accuracy_multi:.2f}%")


**SCI-KIT Implementation of multi-class SVM**

In [ ]:
import numpy as np

# Initialize empty arrays
X_train_sk, Y_train_sk = None, None

for i in range(11):  # Assuming 11 classes
    X, Y = preprocess_image(train_dirs[i], i)  # Get processed images and labels

    # Concatenate iteratively
    if X_train_sk is None:
        X_train_sk, Y_train_sk = X, Y  # First batch initializes arrays
    else:
        X_train_sk = np.concatenate((X_train_sk, X), axis=0)
        Y_train_sk = np.concatenate((Y_train_sk, Y), axis=0)

# Flatten labels to 1D array
Y_train_sk = Y_train_sk.ravel()

In [ ]:
import time
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svm_model_sk = SVC(kernel='rbf', C=1.0, gamma=0.001, decision_function_shape='ovo') 
start_time = time.time()
svm_model_sk.fit(X_train_sk, Y_train_sk)
training_time_sk = time.time() - start_time  # Measure training time
print(f"Training Time: {training_time_sk:.4f} seconds")


In [ ]:
# Predict on test set
Y_pred = svm_model_sk.predict(X_test)

# Compute test accuracy
test_accuracy_sk = accuracy_score(Y_test, Y_pred)

print(f"Test Set Accuracy: {test_accuracy_sk * 100:.2f}%")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Compute confusion matrices
conf_matrix_cvx = confusion_matrix(Y_test, final_predictions)
conf_matrix_svm = confusion_matrix(Y_test, Y_pred)

# Function to plot confusion matrix
def plot_confusion_matrix(cm, class_map, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_map.keys(), yticklabels=class_map.keys())
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(title)
    plt.show()

# Plot confusion matrices
plot_confusion_matrix(conf_matrix_cvx, test_map, "Confusion Matrix - CVXOPT")
plot_confusion_matrix(conf_matrix_svm, test_map, "Confusion Matrix - LIBSVM (Scikit-Learn)")

In [ ]:
# Identify most frequently misclassified classes
def analyze_misclassifications(conf_matrix, class_map):
    misclassified = conf_matrix.copy()
    np.fill_diagonal(misclassified, 0)  # Remove correct classifications
    max_misclassified = np.unravel_index(np.argmax(misclassified), misclassified.shape)
    
    true_class = list(class_map.keys())[max_misclassified[0]]
    predicted_class = list(class_map.keys())[max_misclassified[1]]
    misclassified_count = misclassified[max_misclassified]

    print(f"Most misclassified class: {true_class} → {predicted_class} ({misclassified_count} times)")
    return max_misclassified

# Analyze misclassifications
print("CVXOPT Misclassification Analysis:")
cvx_misclassified = analyze_misclassifications(conf_matrix_cvx, test_map)

print("\nLIBSVM Misclassification Analysis:")
svm_misclassified = analyze_misclassifications(conf_matrix_svm, test_map)

In [ ]:
# Visualize 10 examples of misclassified objects
def visualize_misclassified_examples(X_test, Y_test, Y_pred, true_class_idx, pred_class_idx, class_map, num_samples=10):
    misclassified_indices = np.where((Y_test == true_class_idx) & (Y_pred == pred_class_idx))[0]

    if len(misclassified_indices) > num_samples:
        misclassified_indices = np.random.choice(misclassified_indices, num_samples, replace=False)

    fig, axes = plt.subplots(1, len(misclassified_indices), figsize=(15, 5))
    for i, idx in enumerate(misclassified_indices):
        axes[i].imshow(X_test[idx].reshape(28, 28), cmap="gray")  # Adjust reshape based on dataset
        axes[i].set_title(f"True: {class_map[Y_test[idx]]}\nPred: {class_map[Y_pred[idx]]}")
        axes[i].axis("off")

    plt.show()

# Visualize misclassified examples for CVXOPT
print("Misclassified examples from CVXOPT:")
visualize_misclassified_examples(X_test, Y_test, final_predictions, cvx_misclassified[0], cvx_misclassified[1], test_map)

# Visualize misclassified examples for LIBSVM
print("Misclassified examples from LIBSVM:")
visualize_misclassified_examples(X_test, Y_test, Y_pred, svm_misclassified[0], svm_misclassified[1], test_map)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

# Assuming X_train_sk, Y_train_sk, X_test, Y_test are already defined

# Shuffle the training data to ensure randomness
X_train, Y_train = shuffle(X_train_sk, Y_train_sk, random_state=42)

# Define values of C to test
C_values = [1e-5, 1e-3, 1, 5, 10]
cv_scores = []
test_scores = []

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for C in C_values:
    svm = SVC(C=C, gamma=0.001, kernel='rbf', decision_function_shape='ovo')  # One-vs-One strategy
    
    # Compute cross-validation accuracy
    scores = cross_val_score(svm, X_train, Y_train, cv=kf, scoring='accuracy')
    mean_cv_score = scores.mean()
    cv_scores.append(mean_cv_score)
    
    # Train model on entire training set
    svm.fit(X_train, Y_train)
    
    # Compute test accuracy
    test_pred = svm.predict(X_test)
    test_accuracy = accuracy_score(Y_test, test_pred)
    test_scores.append(test_accuracy)
    
    print(f'C={C}: CV Accuracy={mean_cv_score:.4f}, Test Accuracy={test_accuracy:.4f}')

# Plot results
plt.figure(figsize=(8, 6))
plt.plot(C_values, cv_scores, marker='o', label='5-Fold CV Accuracy')
plt.plot(C_values, test_scores, marker='s', label='Test Accuracy')
plt.xscale('log')  # Log scale for better visualization
plt.xlabel('C (log scale)')
plt.ylabel('Accuracy')
plt.title('Cross-validation & Test Accuracy vs C')
plt.legend()
plt.grid(True)
plt.show()

# Find the best C based on CV accuracy
best_C = C_values[np.argmax(cv_scores)]
print(f'Best C from CV: {best_C}')

# Train final model using best C
final_svm = SVC(C=best_C, gamma=0.001, kernel='rbf', decision_function_shape='ovo')  # One-vs-One strategy
final_svm.fit(X_train, Y_train)
final_test_pred = final_svm.predict(X_test)
final_test_accuracy = accuracy_score(Y_test, final_test_pred)
print(f'Final model test accuracy with C={best_C}: {final_test_accuracy:.4f}')